In [1]:
import boto3
import struct
import re
import subprocess
import json
from shapely.geometry import box, mapping
import geopandas as gpd
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from shapely.geometry import Point
import os
from concurrent.futures import ThreadPoolExecutor

In [2]:
trajectory_file='output/panorama/FramePosOptimised-wagga-wagga_EPSG7844.geojson' # trajectory file
building_points_file='input/Final_Wagga_training_samples.geojson' # copy training building points samples from local computer if needed through VS Code remote explorer
out_epsg='28355'

In [3]:
gdf_buildings=gpd.read_file(building_points_file).to_crs(out_epsg)
gdf_buildings.head()

,WALL_M,STEPS,USAGE,STOREYS,ASSESSOR,address,Area_1,PMF,Dep_500,Dep_200,...,Ground_Lev,Floor_Leve,AGE,UFI,ZONE_DESCR,Floor_height,Ground_surveyed,cluster,Ground_Level_bin,geometry
0,Block,1,Commercial,1,ROBERT,5 MORTIMER PLACE,384.487102,1.5743,-9999.0,-9999.0,...,182.029,182.309,< 1960,676,Light Industrial,0.28,0,0,High,POINT (531615.285 6113336.938)
1,Block,1,Commercial,1,ROBERT,14 MORTIMER PLACE,371.713376,2.3243,-9999.0,-9999.0,...,180.731,181.011,< 1960,1024,Light Industrial,0.28,0,0,High,POINT (531513.659 6113428.854)
2,Brick,1,Commercial,1,ROBERT,9 DOBNEY AVENUE,445.100381,3.7358,-9999.0,-9999.0,...,178.885,179.165,< 1960,1088,Business Development,0.28,0,0,Low,POINT (531728.113 6113930.311)
3,Brick,1,Commercial,2,ROBERT,23 DOBNEY AVENUE,2408.635060,3.4951,-9999.0,-9999.0,...,179.263,179.543,< 1960,1213,Business Development,0.28,0,0,Medium,POINT (531641.524 6113866.683)
4,Brick,4,Commercial,1,ROBERT,300 EDWARD STREET,322.721370,3.5144,-9999.0,-9999.0,...,180.665,180.945,< 1960,1521,Medium Density Residential,0.28,0,0,High,POINT (532264.084 6113914.605)


In [4]:
gdf_panorama = gpd.read_file(trajectory_file).to_crs(out_epsg)
gdf_panorama.head()

,ucid,dRMS,Systemtime_sec,Frame_index,LTP_x_m,LTP_y_m,LTP_z_m,Roll_deg,Pitch_deg,Heading_deg,FrameID,Frame_filename,geometry
0,01846-1739250264,0.009,1739250270,2,534846.1580,6116350.282,181.4110,0.976125,0.718661,-55.304498,1739250264-0000000002,0093_1739250264-0000000002_Panorama_20250211_G...,POINT Z (534845.621 6116348.842 196.084)
1,01846-1739250264,0.009,1739250271,3,534847.0133,6116350.929,181.3396,1.016144,-0.888642,-54.929370,1739250264-0000000003,0093_1739250264-0000000003_Panorama_20250211_G...,POINT Z (534846.471 6116349.493 196.012)
2,01846-1739250264,0.009,1739250271,4,534847.7349,6116351.411,181.3771,1.149973,-0.308095,-54.755426,1739250264-0000000004,0093_1739250264-0000000004_Panorama_20250211_G...,POINT Z (534847.193 6116349.967 196.05)
3,01846-1739250264,0.009,1739250271,5,534848.4947,6116351.954,181.3985,0.972546,-0.311897,-54.410730,1739250264-0000000005,0093_1739250264-0000000005_Panorama_20250211_G...,POINT Z (534847.96 6116350.519 196.071)
4,01846-1739250264,0.009,1739250272,6,534849.2954,6116352.566,181.3863,0.805652,-0.983586,-54.172080,1739250264-0000000006,0093_1739250264-0000000006_Panorama_20250211_G...,POINT Z (534848.756 6116351.126 196.059)


In [5]:
# identify nearest panorama images
# Perform the spatial join with a 40-meter distance tolerance
nearby_points = gpd.sjoin_nearest(gdf_buildings, gdf_panorama, max_distance=40, how='inner', distance_col="distance")
nearby_points

,WALL_M,STEPS,USAGE,STOREYS,ASSESSOR,address,Area_1,PMF,Dep_500,Dep_200,...,Frame_index,LTP_x_m,LTP_y_m,LTP_z_m,Roll_deg,Pitch_deg,Heading_deg,FrameID,Frame_filename,distance
4,Brick,4,Commercial,1,ROBERT,300 EDWARD STREET,322.721370,3.5144,-9999.0000,-9999.0000,...,2697,532287.0254,6113911.892,182.9234,-0.970157,-0.438268,-9.655615,1739412371-0000002697,0630_1739412371-0000002697_Panorama_20250213_G...,22.783669
6,Brick,1,Residential,1,ROBERT,28 SPRING STREET,128.806974,6.9281,2.1277,-9999.0000,...,3052,532008.2113,6114398.715,180.0966,-0.343451,-1.215830,166.445703,1739405253-0000003052,0504_1739405253-0000003052_Panorama_20250213_G...,18.901973
7,Brick,2,Residential,1,ROBERT,9 NORTH PARADE,154.755345,6.5745,-9999.0000,-9999.0000,...,3272,532188.2897,6114311.309,180.7244,1.000789,-0.061197,-101.069299,1739405253-0000003272,0505_1739405253-0000003272_Panorama_20250213_G...,19.948013
8,Brick,1,Residential,2,ROBERT,4 THOMAS STREET,294.580219,6.8309,2.1030,1.1167,...,3367,531982.4553,6114575.693,180.1386,0.110114,-0.999721,19.648854,1739332533-0000003367,0448_1739332533-0000003367_Panorama_20250212_G...,21.276532
9,Brick,1,Residential,1,ROBERT,40 SLOCUM STREET,290.352249,6.2612,1.6366,0.5181,...,6081,532022.1804,6115201.164,180.9586,-0.614706,-0.105462,-71.002953,1739332533-0000006081,0274_1739332533-0000006081_Panorama_20250212_G...,12.458696
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
833,Timber,1,Residential,1,NEIL,5 MILL STREET,81.905406,6.0407,2.0997,1.6591,...,5389,535021.5178,6116317.635,181.9169,2.054471,-0.530138,91.637700,1739251973-0000005389,0095_1739251973-0000005389_Panorama_20250211_G...,25.685563
834,Timber,3,Residential,1,NEIL,2 HENRY STREET,81.609608,5.3563,1.6706,1.3522,...,560,534261.2489,6116060.817,181.4307,-0.730935,-0.272334,114.347479,1739319826-0000000560,0154_1739319826-0000000560_Panorama_20250211_G...,9.052644
835,Timber,5,Residential,1,NEIL,52 HAMPDEN AVENUE,125.146642,5.8437,-9999.0000,-9999.0000,...,160,534521.6238,6116114.263,181.9101,-2.007566,-0.151236,-62.891782,1739319826-0000000160,0135_1739319826-0000000160_Panorama_20250211_G...,26.240996
836,Timber,3,Residential,1,NEIL,6 HAMPDEN AVENUE,146.862693,5.1850,1.3211,-9999.0000,...,931,534234.6463,6115964.543,181.9605,-1.568997,-2.144921,-35.047900,1739318191-0000000931,0176_1739318191-0000000931_Panorama_20250211_G...,22.732602


In [6]:
panorama_names=nearby_points['Frame_filename'].to_list()
panorama_names

['0630_1739412371-0000002697_Panorama_20250213_GDA2020_HarrisonAvenue.jpg',
 '0504_1739405253-0000003052_Panorama_20250213_GDA2020_WestParade.jpg',
 '0505_1739405253-0000003272_Panorama_20250213_GDA2020_NorthParade.jpg',
 '0448_1739332533-0000003367_Panorama_20250212_GDA2020_ThomasStreet.jpg',
 '0274_1739332533-0000006081_Panorama_20250212_GDA2020_CroweLane.jpg',
 '0344_1739332533-0000007076_Panorama_20250212_GDA2020_EvansStreet.jpg',
 '0402_1739331138-0000003877_Panorama_20250212_GDA2020_RhodaAvenue.jpg',
 '0423_1739332533-0000001597_Panorama_20250211_GDA2020_KincaidStreet.jpg',
 '0344_1739332533-0000006771_Panorama_20250212_GDA2020_EvansStreet.jpg',
 '0316_1739332533-0000007005_Panorama_20250212_GDA2020_EvansStreet.jpg',
 '0474_1739404107-0000001010_Panorama_20250213_GDA2020_SpringStreet.jpg',
 '0402_1739331138-0000003886_Panorama_20250212_GDA2020_RhodaAvenue.jpg',
 '0449_1739333728-0000000598_Panorama_20250212_GDA2020_CampbellPlace.jpg',
 '0626_1739411178-0000001068_Panorama_2025021

In [7]:
mfa_token = '858584' # update every time you run
cmd = (f'aws sts get-session-token --serial-number arn:aws:iam::693903849513:mfa/Lavender_AWS_MFA  --token-code {mfa_token}').split()
result = subprocess.run(cmd, capture_output=True)
# Check if the command was successful
if result.returncode == 0:
    # Parse JSON output into a dictionary
    data = json.loads(result.stdout)
    #print(data)  # Print or process the dictionary
else:
    print("Error:", result.stderr)

session = boto3.session.Session(
    aws_access_key_id=data['Credentials']['AccessKeyId'],
    aws_secret_access_key=data['Credentials']['SecretAccessKey'],
    aws_session_token=data['Credentials']['SessionToken']
)
# Initialize an S3 client
s3_client = session.client('s3')

In [8]:
s3_bucket = 'frontiersi-p127-floor-height-woolpert'
s3_prefix='01_WaggaWagga/01_StreetViewImagery/'
download_dir='/mnt/floorheightvolume/panorama_Wagga'
os.makedirs(download_dir, exist_ok=True)

In [9]:
# panorama_names_set=set(panorama_names)  # Convert file_names to a set for faster lookup
# extract frame ids in case not exact match
frame_ids=[panorama_name.split('_')[1] for panorama_name in panorama_names]
frame_ids_set=set(frame_ids)

In [10]:
paginator = s3_client.get_paginator('list_objects_v2')
page_iterator = paginator.paginate(Bucket=s3_bucket, Prefix=s3_prefix)

In [11]:
# Function to download a single file
def download_file(obj_key):
    file_name = os.path.basename(obj_key)
    local_path = os.path.join(download_dir, file_name)
    s3_client.download_file(s3_bucket, obj_key, local_path)
    print(f"Downloaded: {file_name}")

In [16]:
# Use ThreadPoolExecutor for concurrent downloads
with ThreadPoolExecutor(max_workers=10) as executor:
    for page in page_iterator:
        if 'Contents' in page:
            for obj in page['Contents']:
                obj_key = obj['Key']
                file_name = os.path.basename(obj_key)
                if file_name.endswith('.jpg'):
                    frame_id=file_name.split('_')[1]
                    # if file_name in panorama_names_set:# exact file name matching
                    if frame_id in frame_ids_set:
                        # Submit download task to the thread pool
                        executor.submit(download_file, obj_key)
                        # panorama_names_set.remove(file_name)  # Remove to avoid redundant searches
                        frame_ids_set.remove(frame_id)

# # Check if any files were not found
# if panorama_names_set:
#     print(f"Files not found: {panorama_names_set}")

# Check if any files were not found
if frame_ids_set:
    print(f"Files not found: {frame_ids_set}")

Downloaded: 0021_1739250264-0000006702_Panorama_20250211_GDA2020_RowanStreet.jpg
Downloaded: 0049_1739250264-0000001803_Panorama_20250211_GDA2020_HurstStreet.jpg
Downloaded: 0057_1739250264-0000004362_Panorama_20250211_GDA2020_WilliamStreet.jpg
Downloaded: 0061_1739250264-0000005280_Panorama_20250211_GDA2020_MarahStreet.jpg
Downloaded: 0062_1739250264-0000001695_Panorama_20250211_GDA2020_HayesStreet.jpg
Downloaded: 0063_1739250264-0000000256_Panorama_20250211_GDA2020_HayesStreet.jpg
Downloaded: 0063_1739250264-0000000267_Panorama_20250211_GDA2020_HayesStreet.jpg
Downloaded: 0063_1739250264-0000001135_Panorama_20250211_GDA2020_HayesStreet.jpg
Downloaded: 0088_1739250264-0000004482_Panorama_20250211_GDA2020_WilliamStreet.jpg
Downloaded: 0088_1739250264-0000004454_Panorama_20250211_GDA2020_WilliamStreet.jpg
Downloaded: 0091_1739250264-0000003526_Panorama_20250211_GDA2020_ElizabethStreet.jpg
Downloaded: 0111_1739250264-0000002964_Panorama_20250211_GDA2020_WilliamStreet.jpg
Downloaded: 0111

In [17]:
# panorama_names_set

In [18]:
frame_ids_set

set()